In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd
df = pd.read_csv('/content/gdrive/My Drive/Mestrado/second_level.csv')

In [ ]:
import gensim
import gensim.downloader
#embedding = gensim.downloader.load('glove-wiki-gigaword-300')

In [ ]:
from nltk.tokenize import TweetTokenizer
tk = TweetTokenizer()

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize 

def get_embedding_result(word, embedding):
  return embedding[word] if word in embedding else 1


def get_gloVe(X_train, X_test, embeddings_index):
  X_train_vector = []
  for text in X_train:
    words =  [get_embedding_result(word, embeddings_index) for word in word_tokenize(text)]
    X_train_vector.append(np.average(words, axis = 0))

  X_test_vector = []
  for text in X_test:
    words =  [get_embedding_result(word, embeddings_index) for word in word_tokenize(text)]
    X_test_vector.append(np.average(words, axis = 0))


  return X_train_vector, X_test_vector


from sklearn.feature_extraction.text import TfidfVectorizer

def get_gloVe_tf_idf(X_train, X_test, embeddings_index):
  tfidf = TfidfVectorizer()
  tfidf.fit(X_train)
  idf_dict = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

  train_vector = []
  for text in X_train:
    weights = [idf_dict.get(word, 1) for word in word_tokenize(text)]
    words =  [get_embedding_result(word, embeddings_index) for word in word_tokenize(text)]
    train_vector.append(np.average(words, axis = 0, weights = weights))

  test_vector = []
  for text in X_test:
    weights = [idf_dict.get(word, 1) for word in word_tokenize(text)]
    words =  [get_embedding_result(word, embeddings_index) for word in word_tokenize(text)]
    test_vector.append(np.average(words, axis = 0, weights = weights))

  return train_vector, test_vector


In [ ]:

from sklearn.preprocessing import MultiLabelBinarizer

def run(col):
  #### one hote enconding on the labels ########
  mlb = MultiLabelBinarizer()
  y = mlb.fit_transform(df['labels'])

  #### get the text column

  X = df[col]
  import numpy as np
  from sklearn.feature_extraction.text import TfidfTransformer
  from imblearn.pipeline import Pipeline
  from sklearn.feature_extraction.text import CountVectorizer
  from sklearn.multiclass import OneVsRestClassifier ### does not take into account label correlations
  from sklearn.svm import SVC
  from sklearn.metrics import classification_report, multilabel_confusion_matrix, accuracy_score, hamming_loss, jaccard_score

  from sklearn.feature_selection import SelectKBest
  from sklearn.feature_selection import chi2

  #from sklearn.multioutput import ClassifierChain

  from sklearn.utils import compute_class_weight
  from keras.wrappers.scikit_learn import KerasClassifier

  from sklearn.model_selection import GridSearchCV
  from sklearn.naive_bayes import GaussianNB
  from sklearn.ensemble import RandomForestClassifier
  from sklearn.multiclass import OneVsRestClassifier

  from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
  msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

  for train_index, test_index in msss.split(np.array(X), np.array(y)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    text_clf = Pipeline([
                ('vect', CountVectorizer(min_df=20 ,tokenizer=wordpunct_tokenize, stop_words=["(", ")"])),
                ('tfidf', TfidfTransformer()),
                #('smote', MLSmote()),
                #('fs', SelectKBest(chi2, k=500)),
                #('dense', DenseTransformer()),
                ('clf', OneVsRestClassifier(SVC(class_weight="balanced", kernel="sigmoid"))),
                #('clf', RandomForestClassifier(class_weight="balanced"))
            ])
    
    '''
    X_train, X_test = get_gloVe_tf_idf(X_train, X_test, embedding)
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    text_clf = OneVsRestClassifier(SVC(class_weight="balanced", kernel="sigmoid"))
    '''
    

    text_clf.fit(X_train, y_train)

    y_pred = text_clf.predict(X_test)

    from sklearn.metrics import classification_report, accuracy_score, multilabel_confusion_matrix, jaccard_score

    print(accuracy_score(y_test, y_pred))
    print(jaccard_score(y_test, y_pred, average = "weighted"))
    print(classification_report(y_test, y_pred))
    print(multilabel_confusion_matrix(y_test, y_pred))


In [3]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import unidecode

df['text'] = df['text'].apply(unidecode.unidecode)

In [ ]:
#df[['text', 'labels', 'text_processed']].to_csv('pre_processing_important.csv')

In [ ]:
# adding dose got better

In [5]:
# Thanks : https://github.com/NeelShah18/emot/blob/master/emot/emo_unicode.py
EMOTICONS = {
    u":‑)":"happy",
    u":-D":"happy",
    u":)":"happy",
    u":-]":"happy",
    u":]":"happy",
    u":-3":"  happy",
    u":3":"happy",
    u":->":"happy",
    u":>":"happy",
    u"8-)":"happy",
    u":o)":"happy",
    u":-}":"happy",
    u":}":"happy",
    u":-)":"happy",
    u":c)":"happy",
    u":^)":"happy",
    u"=]":"happy",
    u"=)":"happy",
    u":‑D":"laughing",
    u":D":"laughing",
    u"8‑D":"laughing",
    u"8D":"laughing",
    u"X‑D":"laughing",
    u"XD":"laughing",
    u"=D":"laughing",
    u"=3":"laughing",
    u"B^D":"laughing",
    u":-))":"very happy",
    u":‑(":"sad",
    u":-(":"sad",
    u":(":"sad",
    u":‑c":"sad",
    u":c":"sad",
    u":‑<":"sad",
    u":<":"sad",
    u":‑[":"sad",
    u":[":"sad",
    u":-||":"sad",
    u">:[":"sad",
    u":{":"sad",
    u":@":"sad",
    u">:(":"sad",
    u":'‑(":"crying",
    u":'(":"crying",
    u":'‑)":"very happy",
    u":')":"very happy",
    u"D‑':":"horror",
    u"D:<":"disgust",
    u"D:":"sadness",
    u"D8":"dismay",
    u"D;":"dismay",
    u"D=":"dismay",
    u"DX":"dismay",
    u":‑O":"surprise",
    u":O":"surprise",
    u":‑o":"surprise",
    u":o":"surprise",
    u":-0":"shock",
    u"8‑0":"yawn",
    u">:O":"yawn",
    u":-*":"kiss",
    u":*":"kiss",
    u":X":"kiss",
    u";‑)":"smirk",
    u";)":"smirk",
    u"*-)":"smirk",
    u"*)":"smirk",
    u";‑]":"smirk",
    u";]":"smirk",
    u";^)":"smirk",
    u":‑,":"smirk",
    u";D":"smirk",
    u":‑P":"playful",
    u":P":"playful",
    u"X‑P":"playful",
    u"XP":"playful",
    u":‑Þ":"playful",
    u":Þ":"playful",
    u":b":"playful",
    u"d:":"playful",
    u"=p":"playful",
    u">:P":"playful",
    u":‑/":"uneasy",
    u":/":"uneasy",
    u":-[.]":"uneasy",
    u">:[()]":"uneasy",
    u">:/":"uneasy",
    u":[()]":"uneasy",
    u"=/":"uneasy",
    u"=[()]":"uneasy",
    u":L":"uneasy",
    u"=L":"uneasy",
    u":S":"uneasy",
    u":‑|":"emotionless",
    u":|":"emotionless",
    u":$":"embarrassed",
    u":‑x":"tongue-tied",
    u":x":"tongue-tied",
    u":‑#":"tongue-tied",
    u":#":"tongue-tied",
    u":‑&":"tongue-tied",
    u":&":"tongue-tied",
    u"O:‑)":"innocent",
    u"O:)":"innocent",
    u"0:‑3":"innocent",
    u"0:3":"innocent",
    u"0:‑)":"innocent",
    u"0:)":"innocent",
    u":‑b":"playful",
    u"0;^)":"innocent",
    u">:‑)":"evil",
    u">:)":"evil",
    u"}:‑)":"evil",
    u"}:)":"evil",
    u"3:‑)":"evil",
    u"3:)":"evil",
    u">;)":"evil",
    u"|;‑)":"cool",
    u"|‑O":"bored",
    u":‑J":"tongue-in-cheek",
    u"#‑)":"party",
    u"%‑)":"confused",
    u"%)":"confused",
    u":-###..":"sick",
    u":###..":"sick",
    u"<:‑|":"dump",
    u"(>_<)":"troubled",
    u"(>_<)>":"troubled",
    u"(';'\)":"baby",
    u"\(\^\^>``":"nervous",
    u"\(\^_\^;)":"nervous",
    u"\(-_-;\)":"nervous",
    u"\(~_~;\) \(・\.・;\)":"nervous",
    u"\(-_-\)zzz":"sleeping",
    u"\(\^_-\)":"wink",
    u"((+_+))":"confused",
    u"\(\+o\+\)":"confused",
    u"\(o\|o\)":"ultraman",
    u"\^_\^":"joyful",
    u"\(\^_\^\)/":"joyful",
    u"\(\^O\^\)／":"joyful",
    u"\(\^o\^\)／":"joyful",
    u"\(__\)":"respect",
    u"_(._.)_":"respect",
    u"<(_ _)>":"respect",
    u"<m(__)m>":"respect",
    u"m(__)m":"respect",
    u"m(_ _)m":"respect",
    u"('_')":"crying",
    u"(/_;)":"crying",
    u"(T_T) (;_;)":"crying",
    u"(;_;":"Sad of Crying",
    u"(;_:)":"crying",
    u"(;O;)":"crying",
    u"(:_;)":"crying",
    u"(ToT)":"crying",
    u";_;":"crying",
    u";-;":"crying",
    u";n;":"crying",
    u";;":"crying",
    u"Q.Q":"crying",
    u"T.T":"crying",
    u"QQ":"crying",
    u"Q_Q":"crying",
    u"(-.-)":"shame",
    u"(-_-)":"shame",
    u"(一一)":"shame",
    u"(；一_一)":"shame",
    u"(=_=)":"tired",
    u"(=^·^=)":"cat",
    u"(=^··^=)":"cat",
    u"=_^=	":"cat",
    u"(..)":"looking down",
    u"(._.)":"looking down",
    u"^m^":"giggling",
    u"(・・?":"confusion",
    u"(?_?)":"confusion",
    u">^_^<":"laugh",
    u"<^!^>":"laugh",
    u"^/^":"laugh",
    u"（*^_^*）" :"laugh",
    u"(^<^) (^.^)":"laugh",
    u"(^^)":"laugh",
    u"(^.^)":"laugh",
    u"(^_^.)":"laugh",
    u"(^_^)":"laugh",
    u"(^^)":"laugh",
    u"(^J^)":"laugh",
    u"(*^.^*)":"laugh",
    u"(^—^）":"laugh",
    u"(#^.^#)":"laugh",
    u"（^—^）":"waving",
    u"(;_;)/~~~":"waving",
    u"(^.^)/~~~":"waving",
    u"(-_-)/~~~ ($··)/~~~":"waving",
    u"(T_T)/~~~":"waving",
    u"(ToT)/~~~":"waving",
    u"(*^0^*)":"excited",
    u"(*_*)":"amazed",
    u"(*_*;":"amazed",
    u"(+_+) (@_@)":"amazed",
    u"(*^^)v":"cheerful",
    u"(^_^)v":"cheerful",
    u"((d[-_-]b))":"listening to music",
    u'(-"-)':"worried",
    u"(ーー;)":"worried",
    u"(^0_0^\)":"eyeglasses",
    u"(＾ｖ＾)":"happy",
    u"(＾ｕ＾)":"happy",
    u"(^)o(^)":"happy",
    u"(^O^)":"happy",
    u"(^o^)":"happy",
    u")^o^(":"happy",
    u":O o_O":"surprised",
    u"o_0":"surprised",
    u"o.O":"Surpised",
    u"(o.o)":"surprised",
    u"oO":"surprised",
    u"(*￣m￣)":"dissatisfied",
    u"(‘A`)":"deflated",
    u"<3" : "love"
}

In [6]:
def convert_emoticons(text):
  for word, initial in EMOTICONS.items():
    text = text.replace(word.lower(), initial)
  return text


In [7]:
import re

In [8]:
def remove_urls(text):
   vTEXT = re.sub(r'((https|http)?:\/\/)(\w|\.|\/|\?|\=|\&|\%)*\b', '<URL>', text, flags=re.MULTILINE)
   return vTEXT

In [9]:
import spacy
import spacy.cli
spacy.cli.download("en_core_web_lg")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [12]:
ner = spacy.load('en_core_web_lg')
ner.add_pipe('merge_entities')

<function spacy.pipeline.functions.merge_entities>

In [13]:
def ner_check(text):
  doc = ner(text)
  newString = text
  for e in reversed(doc.ents): #reversed to not modify the offsets of other entities when substituting
      if e.label_.lower() not in ['percent', 'date', 'time', 'cardinal', 'gpe', 'quantity', 'ordinal', 'money']:
        continue
      start = e.start_char
      end = start + len(e.text)
      newString = newString[:start] + "<" + e.label_ + ">" + newString[end:]
  return newString

In [ ]:
def delete_words_one(text):
  word_list = text.split(' ')
  result = []
  for word in word_list:
    if len(word) > 1 or word in string.punctuation:
      result.append(word)
  return ' '.join(result)

In [ ]:
#df['del_1word'] = df['removals'].apply(delete_words_one) #does nothing

In [ ]:
run('del_1word')

https://medium.com/@lukei_3514/dealing-with-contractions-in-nlp-d6174300876b

In [14]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.7 MB/s 
     |████████████████████████████████| 596 kB 47.9 MB/s 
     |████████████████████████████████| 86 kB 4.6 MB/s 
     |████████████████████████████████| 6.6 MB 35.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [15]:
! sudo apt install -y openjdk-8-jdk
! sudo update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
! pip install git+https://github.com/MCFreddie777/language-check.git --no-cache-dir
! pip install pycontractions --no-cache-dir

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jdk is already the newest version (8u312-b07-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/MCFreddie777/language-check.git to /tmp/pip-req-build-j5c2p1vb
  Running command git clone -q https://github.com/MCFreddie777/language-check.git /tmp/pip-req-build-j5c2p1vb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
from pycontractions import Contractions
# or specify any model from the gensim.downloader api
cont  = Contractions(api_key="glove-twitter-100")

In [17]:
from bs4 import BeautifulSoup
import re

In [ ]:
("needed!  I'm").split(" ")

In [ ]:
from nltk.tokenize import wordpunct_tokenize, TreebankWordTokenizer
wt = TreebankWordTokenizer()

In [18]:
import spacy.cli

spacy.cli.download("en_core_web_lg")
import spacy
sp_lg = spacy.load('en_core_web_lg')
{(ent.text.strip(), ent.label_) for ent in sp_lg("Your sentence with date here").ents}

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


set()

In [20]:
!pip install scispacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_md-0.5.0.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_md-0.5.0.tar.gz (120.2 MB)


In [22]:
import scispacy
import spacy
nlp = spacy.load("en_ner_bc5cdr_md")

OSError: ignored

In [23]:
from collections import defaultdict

import string
def tag_medicine(text):

  temp = defaultdict()
  d  = sp_lg(text)
  
  for token in d:
    temp[token.idx] = token.pos_
  
  doc = nlp(text)
  disease = []
  chemical = []
  for ent in doc.ents:
    if temp[ent.start_char] == 'NOUN':
      #print(ent.text,"|", ent.label_,"|", temp[ent.start_char])
      #print(token.idx)
      if "(" in ent.text or ")" in ent.text:
        continue
      if ent.label_ == 'DISEASE':
        disease.append(ent.text)
      elif ent.label_ == 'CHEMICAL':
        chemical.append(ent.text)

  if len(disease) == 1:
    text = re.sub(disease[0], "<DISEASE>", text)
  if len(chemical) == 1:
    text = re.sub(chemical[0], "<CHEMICAL>", text)

  if len(disease) > 1:
    text = re.sub("|".join(disease), "<DISEASE>", text)
  if len(chemical) > 1:
    text =  re.sub("|".join(chemical), "<CHEMICAL>", text)

  return text
  

In [ ]:
df['text'][0]

In [ ]:
df['text']

In [ ]:
from spacy.matcher import Matcher

pattern = [{'ENT_TYPE':'CHEMICAL'}, {'LIKE_NUM': True}, {'IS_ASCII': True}]
matcher = Matcher(nlp.vocab)
matcher.add("DRUG_DOSE", [pattern])

def replace_drug_dose(text):
    doc = nlp(text)
    matches = matcher(doc)
    names = []

    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # get string representation
        span = doc[start:end]  # the matched span
        names.append(span.text)
    if len(names) == 0:
      return text
    return re.sub("|".join(names), "<DOSE>", text)


In [ ]:
df['text'][0]

In [ ]:
df['dose'] =  df['text'].apply(replace_drug_dose)


In [ ]:
df['text'][0]

In [ ]:
import string

for x in df['text']:
  for word in x.split(' '):
    for char in word:
      if char in string.punctuation:
        print(word)
        break

In [ ]:
#####################################################
# Remove html tags          
#####################################################
def strip_html_tags(text):
    """remove html tags from text"""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

In [ ]:
######################################################
# Replace emails by an email tag
######################################################
def remove_emails(text):
  return re.sub('[\w\.-]+@[\w\.-]+', '',text)

#######################################################
# Replace urls by an url tag


#######################################################
# remove digits and words with digits
#######################################################
def remove_digit(text):
  return ' '.join(s for s in text.split() if not any(c.isdigit() for c in s))

def remove_mentions(text):
  return re.sub("@[A-Za-z0-9_]+","", text)

def remove_hashtags(text):
  return re.sub("#[A-Za-z0-9_]+","", text)

def remove_phonenumbers(text):
  return re.sub('^(\+\d{1,2}\s)?\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}$', "", text)

In [8]:
#######################################################
# Expand contractions taking into account the context
#######################################################
def expand_contractions(text):
  return list(cont.expand_texts([text]))[0]

In [ ]:
#######################################################
# Removes extra spaces, tabs, etc, being in in the middle
# end or beginning of the text
#######################################################
def remove_extra_spaces(text):
  return " ".join(text.split())

In [10]:
import json
  
# Opening JSON file
f = open('/content/gdrive/My Drive/Mestrado/myslang.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
abbreviations = []
for texto in df['text']:
  for ele in texto.split():
    if ele.upper() in data:
      abbreviations.append(ele)

abbreviations = set(abbreviations)

#for x in abbreviations:
#  print(x,"=>", data[x.upper()])


In [11]:
# btw, idk
def get_abbreviations():
  df = pd.read_csv('/content/gdrive/My Drive/Mestrado/sentiment analysis/abbreviations.txt', delimiter="=", header=None)
  df.columns=['abbr', 'meaning']
  return dict(zip(df.abbr, df.meaning))

In [12]:
abbr = get_abbreviations()
abbr['IDK'] = 'I do not know'


In [ ]:
from collections import Counter
abbreviations = []
for texto in df['text']:
  for ele in texto.split():
    if ele.upper() in abbr:
      abbreviations.append(ele)

print(Counter(abbreviations))
abbreviations = set(abbreviations)

for x in abbreviations:
  print(x,"=>", abbr[x.upper()])

Counter({'u': 8, 'idk': 3, 'lol': 2, 'btw': 1, 'BTW': 1, 'IDK': 1})
u => You
IDK => I do not know
btw => By The Way
BTW => By The Way
idk => I do not know
lol => Laughing Out Loud


In [ ]:
2*2 + 6

In [14]:
#####################################
## replace abbreviations often found
## in social media text
def replace_abbr(text):
  return   " ".join(abbr.get(ele.upper(), ele) for ele in text.split())

In [ ]:
!sudo apt-get install -y swig

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 1s (1,155 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Fr

In [ ]:
!sudo pip install jamspell

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 174 kB 5.1 MB/s 
  Created wheel for jamspell: filename=jamspell-0.0.12-cp37-cp37m-linux_x86_64.whl size=1347595 sha256=6f643a1528b690bec8736ce65486b8897db04d1f7939f0f97b92bb4af1aa718a
  Stored in directory: /root/.cache/pip/wheels/68/df/9c/9b335e69aa0f28e7f508ec0ebefadcc703f168beb52ae7ebe7
Successfully built jamspell


In [ ]:
!wget https://github.com/bakwc/JamSpell-models/raw/master/en.tar.gz

--2022-06-14 20:42:17--  https://github.com/bakwc/JamSpell-models/raw/master/en.tar.gz
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bakwc/JamSpell-models/master/en.tar.gz [following]
--2022-06-14 20:42:17--  https://raw.githubusercontent.com/bakwc/JamSpell-models/master/en.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36611828 (35M) [application/octet-stream]
Saving to: ‘en.tar.gz.1’

en.tar.gz.1         100%[===================>]  34.92M   180MB/s    in 0.2s    

2022-06-14 20:42:17 (180 MB/s) - ‘en.tar.gz.1’ saved [36611828/36611828]



In [ ]:
import jamspell

corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('en.bin')

False

In [ ]:
!tar -xvf en.tar.gz

en.bin


In [ ]:
def spell_correct(text):
  return corrector.FixFragment(text)

In [ ]:
def remove_extra_apostrophe(text):
  return text.replace('"', "")

In [ ]:
remove_extra_apostrophe('She said "what is that"? ')

'She said what is that? '

In [ ]:
def remove_extra_spaces(text):
  return " ".join(text.split())

In [ ]:
### separate punctuation ########
import string
regular_punct = list(string.punctuation)

def spacing_punctuation(text, regular_punct):
    for punc in regular_punct:
        if punc in text:
            text = text.replace(punc, f' {punc} ')
    return text.strip()

spacing_punctuation("Good Morning!! Hoow are you?? I'm fine",regular_punct)

"Good Morning !  !  Hoow are you ?  ?  I ' m fine"

In [ ]:
import nltk
nltk.download('punkt')
np.mean(df['text_corrected'].apply(lambda x : len(wordpunct_tokenize(x))))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


124.57395143487858

In [ ]:
from bs4 import BeautifulSoup

df['text_w_tags'] = df['text'].apply(strip_html_tags)

In [9]:
df['text_contr'] = df['text'].apply(expand_contractions)

[==================================================] 100.0% 387.1/387.1MB downloaded


In [15]:
df['slang'] = df['text_contr'].apply(replace_abbr)

In [ ]:
results = set()
from nltk.tokenize import wordpunct_tokenize, TreebankWordTokenizer
df['text_corrected'].apply(lambda x : wordpunct_tokenize(x)).apply(results.update)
print(results)

print(len( set(results)))

{'reclaim', 'name', 'reaches', 'relax', 'lenient', 'someone', 'Usually', 'overdoses', 'browse', 'Gene', 'elevating', 'masses', 'happiness', 'cleanings', 'relevant', 'zoloft', 'adult', 'RSD', 'changed', 'strengths', 'breath', 'cells', 'appointment', 'Okay', 'isolated', 'Will', 'Brown', 'seed', 'really', 'anorexia', 'element', 'movement', 'seriously', 'lift', 'affecting', 'label', 'reaching', 'Sharing', 'BUN', 'weeks', 'dry', '3cm', 'plain', 'ran', 'searched', 'inguinal', 'giant', 'lows', 'adroitness', 'temazepam', 'opposed', 'grains', 'throw', 'safely', 'Sept', 'unhealthy', 'attemp', 'administration', 'psychiatrist', 'floating', 'anulom', 'sometimes', 'real', 'ms', 'heal', 'oncologist', 'remote', 'Hmm', 'pick', 'isolation', 'viable', 'FEV1', 'gynecologist', 'pretty', 'strongly', 'ten', 'poking', 'demand', 'meaning', 'evil', 'affects', 'sides', 'burning', 'HR', 'left', 'requested', 'impactful', 'done', 'content', 'hygienic', 'splashed', 'specialty', 'puddles', 'dense', 'strips', 'orderly

In [17]:
df['emoticons'] = df['slang'].apply(convert_emoticons)

In [ ]:
df['text_corrected'] = df['emoticons'].apply(spell_correct)

In [ ]:
df['drug_dose'] = df['text_corrected'].apply(replace_drug_dose)

In [ ]:
df['medical_ner'] = df['drug_dose'].apply(tag_medicine)

In [ ]:
df['other_ner'] = df['medical_ner'].apply(ner_check)

In [ ]:
df['removals'] = df['other_ner'].apply(remove_urls)
df['removals'] = df['removals'].apply(remove_hashtags)
df['removals'] = df['removals'].apply(remove_mentions)
df['removals'] = df['removals'].apply(remove_extra_apostrophe)
df['removals'] = df['removals'].apply(remove_phonenumbers)

In [ ]:
df['text_corrected'] = df['removals'].apply(spell_correct)

In [ ]:
run('removals')

In [ ]:
run('text_corrected')

In [ ]:
run('removals')

In [ ]:
run('other_ner')

In [ ]:
run('text')

In [ ]:
df['text_w_tags'] = df['text_w_tags'].apply(remove_digit)

In [ ]:
df['text_w_tags'] = df['text_w_tags'].apply(remove_hashtags)

In [ ]:
df['text_w_tags'] = df['text_w_tags'].apply(remove_mentions)

In [ ]:
df['text_w_tags'] = df['text_w_tags'].apply(remove_phonenumbers)

In [ ]:
df['text_w_tags'] = df['text_w_tags'].apply(remove_emails)

In [ ]:
df['text_w_tags'] = df['text_w_tags'].apply(remove_urls)

In [ ]:
df['text_w_tags'] = df['text_w_tags'].apply(remove_extra_spaces)

In [ ]:
df['text_w_tags'] = df['text_w_tags'].apply(remove_extra_apostrophe)

In [ ]:
df['text_not_abbr'] = df['text_w_tags'].apply(replace_abbr)

In [ ]:
df['text_contr'] = df['text_not_abbr'].apply(expand_contractions)

In [ ]:
df ['text_contr'] = df['text_contr'].apply(lambda x : spacing_punctuation(x, regular_punct))

In [ ]:
df['text_corrected'] = df['text_contr'].apply(spell_correct)

In [ ]:
df['text_corrected'] = df['text_corrected'].apply(remove_extra_spaces)

In [ ]:
import nltk
nltk.download('punkt')
import numpy as np


#np.array(df['to_save'].apply(lambda x : len(x.split(' ')))).mean()
results = set()
#df['to_save'].str.lower().str.split().apply(results.update)
print(len(results))


In [ ]:
df['text_corrected']

In [ ]:
df.to_csv('second_level_pre.csv')

In [ ]:
df['text_not_abbr'], df['text_contr']

In [ ]:
#c = 0
#for x in df['to_save']:
#  for word in x.split(' '):
#    if word == '<entity>':
#      c +=1

In [ ]:
#c

In [ ]:
!apt update
!apt install enchant --fix-missing
!apt install -qq enchant
!pip install pyenchant

In [ ]:
import enchant 
d = enchant.Dict("en_US") 
d.check("y/o")

In [ ]:
words = []
for X in df['text_corrected']: 
  for word in X.split(' '):
    if word != '' and word not in string.punctuation and not d.check(word):
      words.append(word) 

In [ ]:
words = set(words)
len(words)

In [ ]:
!pip install scispacy

In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_md-0.5.0.tar.gz

In [ ]:
df.to_csv('first_task_preprocessing.csv')

In [ ]:
import spacy
import scispacy

from scispacy.linking import EntityLinker

nlp = spacy.load("en_core_sci_sm")



In [ ]:
from collections import defaultdict
found = defaultdict(int)
for word in words:
  found[nlp(word)] += 1


In [ ]:
nlp("ok").ents

In [ ]:
len(nlp("adderall").ents)

In [ ]:
#####################################
## replace abbreviations often found
## in social media text

def check_len(res):
  if res not in words:
    return res
  val = nlp(res).ents
  if (len(val)) > 0:
    return "<ENTITY>"
  else:
    return res

def replace_entities(text):
  return   " ".join(check_len(ele) for ele in text.split())

In [ ]:
df['last_one'] = df['text_corrected'].apply(replace_entities)

In [ ]:
do_not_remove = {"no", "nor", "not", "few", "very",  "don'", "don't",  "'aren'",
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't"}

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop = set(stopwords.words('english')) - do_not_remove
len(stop)

In [ ]:
df['last_one'] = df['last_one'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
df['to_save'] = df['last_one'].str.lower()

In [ ]:
#df.to_csv('NEW_ONE.csv')

In [ ]:
replace_entities(df['text'][102])

In [ ]:
len(found)

In [ ]:
found

In [ ]:

doc = nlp("temapazan")

print(doc.ents)
# Let's look at a random entity!
entity = doc.ents[0]

print("Name: ", entity)
# Each entity is linked to UMLS with a score
# (currently just char-3gram matching).
linker = nlp.get_pipe("scispacy_linker")

for entity in doc.ents:
  for umls_ent in entity._.kb_ents:
    print(linker.kb.cui_to_entity[umls_ent[0]])

In [ ]:
! pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bionlp13cg_md-0.5.0.tar.gz

In [ ]:
! pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz

In [ ]:
import scispacy
import spacy
from spacy import displacy
from collections import Counter
#import en_core_web_sm
import en_ner_bc5cdr_md
import en_core_sci_sm
import en_ner_bionlp13cg_md
from scispacy.abbreviation import AbbreviationDetector
from scispacy.umls_linking import UmlsEntityLinker
from collections import OrderedDict
from pprint import pprint

def display_entities(model,document):
    """ 
    This function displays word entities

    Parameters: 
         model(module): A pretrained model from spaCy(https://spacy.io/models) or ScispaCy(https://allenai.github.io/scispacy/)
         document(str): Document to be processed

    Returns: Image rendering and list of named/unnamed word entities and entity labels 
     """
    nlp = model.load()
    doc = nlp(document)
    displacy_image = displacy.render(doc, jupyter=True,style='ent')
    entity_and_label = pprint(set([(X.text, X.label_) for X in doc.ents]))
    return  displacy_image, entity_and_label

In [ ]:
display_entities(en_core_sci_sm,"temazapan is not that good for you.")

In [ ]:
# number unique words, average document length

def voc_size():
  new= df['text_corrected'].str.split()
  new=new.values.tolist()
  corpus=[word for i in new for word in i]
  corpus = set(corpus)
  print(len(corpus))
  print(corpus)
voc_size()


# https://towardsdatascience.com/text-pre-processing-stop-words-removal-using-different-libraries-f20bac19929a

In [ ]:
! pip install iterative-stratification

In [ ]:
from ast import literal_eval
df['labels'] = df['labels'].apply(literal_eval)